<center>
<img style="float: center;" src="images/CI_horizontal.png" width="400">
</center>
<center>
    <span style="font-size: 1.5em;">
        <a href='https://www.coleridgeinitiative.org'>Website</a>
    </span>
</center>


<center> Julia Lane, Benjamin Feder, Brian Kim, Ekaterina Levitskaya, Allison Nunez. </center>

# Unsupervised Machine Learning

There are problems where the goal is not necessarily prediction, but instead it is to discover any inherent groupings or patterns in the data. Unsupervised machine learning methods can help tackle these situations. Clustering is the most common unsupervised machine learning technique, but other methods such as principal components analysis (PCA) or neural networks implementations (e.g. self-organizing maps) are also widely-used in the field. In fact, you have actually already done a form of unsupervised learning already in the text analysis notebook. Topic modeling with Latent Dirichlet Allocation is a form of unsupervised learning. This notebook will introduce unsupervised machine learning through the lens of a clustering example.

## Introduction to Clustering

Clustering is used to group data points together that are similar to each other. Optimally, a given clustering method will produce groupings with high intra-cluster (within) similarity and low inter-cluster (between) similarity. Clustering algorithms typically require a distance or similarity metric to generate clusters. They take a dataset and a distance metric (and sometimes additional parameters) and they generate clusters based on the distance metric. The most common distance metric is Euclidean distance, but other commonly-used metrics are Manhattan, Minkowski, Chebyshev, Cosine, Hamming, Pearson, and Mahalanobis.

Most clustering algorithms also require the user to specify the number of clusters (or another parameter that indirectly determines the number of clusters) in advance as a parameter. This is often difficult to do a priori and typically makes clustering an iterative and interactive task. Another aspect of clustering that makes it interactive is often the difficulty in automatically evaluating the quality of the clusters. While various analytical clustering metrics have been developed, the best clustering ones are task-dependent and thus must be evaluated by the user. There may be different clusterings that can be generated with the same data. One can imagine clustering similar news stories based on the topic content, based on the writing style, or sentiment. The right set of clusters depends on the user and the task at hand. Clustering is therefore typically used for exploring the data, generating clusters, exploring the clusters, and then re-running the clustering method with different parameters or modifying the clusters (by splitting or merging the previous set of clusters). Interpreting a cluster can be nontrivial: one can look at the centroid of a cluster, frequency distributions of different features (and compare them to the prior distribution of each feature), or other aspects.

This notebook focuses on **K-Means clustering** (*k* defines the number of clusters), which is considered to be the most commonly used clustering method. The algorithm works as follows:
1. Select *k* (the number of clusters to generate).
2. Initialize the algorithm by selecting k points as centroids of the *k* clusters. This is typically done by selecting k points uniformly at random.
3. Assign each point a cluster according to the nearest centroid.
4. Recalculate cluster centroids based on the assignment in **(3)** as the mean of all data points belonging to that cluster.
5. Repeat **(3)** and **(4)** until convergence.

The algorithm stops when the assignments do not change from one iteration to the next. The final set of clusters, however, depends on the starting points. If initialized differently, it is possible to obtain different clusters. One common practical trick is to run *k*-means several times, each with different (random) starting points. The *k*-means algorithm is fast, simple, and easy to use and is often a good first clustering algorithm to try and see if it fits one's needs. When the mean of the data points cannot be computed (i.e. for categorical variables), a related method called *K-medoids* can be used.

### Learning Objectives

This notebook demonstrates how *k*-means clustering can be employed to better understand the types of PhD students based on funding history. This notebook covers a few different values of *k* to see how to best characterize the PhD funding experiences by looking for differentiation between each of the clusters.

## Import Packages and Set Up


The primary R package used in this notebook for clustering is called `cluster`. The usual packages for database connection and data manipulation/visualization will also be imported.

In [ ]:
#database interaction imports
library(odbc)

# for data manipulation/visualization
library(tidyverse)
library(ggplot2)

# clustering
library(cluster)
library(reshape2)

In [ ]:
# Connect to the database
con <- DBI::dbConnect(odbc::odbc(),
                     Driver = "SQL Server",
                     Server = "msssql01.c7bdq4o2yhxo.us-gov-west-1.rds.amazonaws.com",
                     Trusted_Connection = "True")

## 1. Read in the Data

To start the clustering example, the proper dataset must be crafted. In this example, the dataset will contain funding history from UMETRICS, as well as time measures regarding degree completion from SED. The individuals in this dataset must individually appear in both the SED and UMETRICS tables.

In [ ]:
# read into R
qry <- "
select sed.drf_id, team_size, any_non_federal
from ds_nsf_ncses.dbo.nsf_sed sed
join tr_uncf_excelencia.dbo.sed_umetrics_xwalk xwalk
on sed.drf_id = xwalk.drf_id
join ds_iris_umetrics.dbo.semester sem
on xwalk.emp_number = sem.emp_number
where sed.phdfy = '2015' 
"
students <- dbGetQuery(con, qry)

# see the dataframe
head(students)

Add total number of semesters of funding:

In [ ]:
# Add total number of semesters of funding
students <- students %>%
    group_by(drf_id) %>%
    mutate(sem_funding = n())  # count the number of rows per each unique drf_id

students %>% head()

Add a flag for federal and non-federal funding:

In [ ]:
# If team_size > 0, then flag as federal (add 1), if team_size is missing, then add 1 in the "any_non_federal" variable
students <- students %>%
    mutate(federal = ifelse(team_size > 0, 1, 0),
           non_federal = ifelse(is.na(team_size) | any_non_federal == 1, 1, 0))

head(students)

Count total number of semesters with federal and non-federal funding:

In [ ]:
students <- students %>%
            group_by(drf_id) %>%                                 # for each unique drf_id
            mutate(sem_federal = sum(federal, na.rm = TRUE),     # sum values in the federal flag
                   sem_non_federal = sum(non_federal)) %>%       # sum values in the any_non_federal flag
            select(-c(team_size, any_non_federal, federal, non_federal)) %>%  # remove these columns
            slice(1) %>%                                         # deduplicate rows, choose just 1 row per unique drf_id
            replace(is.na(.), 0)                                 # fill NA with 0

head(students)

## 2. Clean the Data

### Missing Data

Before running k-means clustering on `students`, the data frame must be further cleaned. Clustering algorithms will not work if there are any missing values in any of the features. Here, the `na.omit()` function removes all rows with any NA values. (If a student has missing information in any of the columns, a row will be dropped).

> Note: **never remove data** if possible - in a real world setting any missing data would be likely filled with an imputation or baseline assumption. Imputation of missing data will be discussed during the Inference session.

In [ ]:
# Check number of rows (where each row is a unique student)
nrow(students)
head(students)

In [ ]:
# Remove all missing data points before running clustering
# na.omit will remove any rows with any NA values
students_ml <- na.omit(students)

In [ ]:
# Check number of rows after dropping rows with any NA values
nrow(students_ml)

### Explanatory power

 The `drf_id` variable should be removed from the data frame since the feature does not provide any explanatory power for the k-means algorithm.

In [ ]:
# Remove drf_id
students_features <- students_ml %>%
    ungroup() %>%
    select(-drf_id)

In [ ]:
head(students_features)

### Variable Types

 Recall that k-means algorithms only work properly with continuous features. This is because k-means calculates its distance measure using euclidean distance, which is the distance between each data point and the centroid of a cluster. It is hard to assign positions for categorical variables in the euclidean space. 

> There are more sophisticated clustering algorithms that do not use Euclidean distances and thus allow categorical variables in the model, such as k-medoids as mentioned in the Introduction to Clustering [section](#Introduction-to-Clustering). R packages such as `klaR` and `cba` can be used to implement algorithms to handle categorical variables.

In [ ]:
# Check data type of all variables - make sure all of them are numeric
str(students_features)

#### Scaling Numerical Features

It is important to consider scaling these features before computing *k*-means clustering, especially if the metrics are on a variety of numerical scales. The scales of each of the variables can be verified using `summary`.

In [ ]:
# Get descriptions of each variable using "summary" function
summary(students_features)

If the variables are on different numerical scales, the `scale()` function can be used to set all of these variables onto the same scale.

> Note: It is not always the case that the variables will be on different scales. Exercise caution before implementing any variable scaling.

In this case the variables are comparable, as all of them represent semesters of funding, therefore, the code below is commented out.

In [ ]:
# Here is the code for scaling of the features, if needed
#students_features <- scale(students_features)

# View first rows after scaling
#head(students_features)

The data frame is now ready for clustering.

## 3. Choose the Number of Clusters, *K*

Running a *k*-means model is simple once the underlying dataset is ready: the `kmeans()` function will implement the algorithm as long as the number of clusters (called `centers`) is specified. What k value is optimal when starting? With multiple features, it is hard to visualize the data and decide the proper number by using the eyes. K will be set to a low number, such as 3, to see these results.  

Because *k*-means clustering will generate different results (due to different starting points), set a seed so that the work in this notebook can be reproducible using `set.seed`. To get the same results, the same seed must be set before running the clustering algorithm every time. Luckily, if the same seed is set and all collaborators are running the same *k*-means algorithm, the results will be consistent, even if collaborators are working in different environments, i.e. Jupyter notebooks.

### k = 3

In [ ]:
# Initialize the model and run on students_features
set.seed(1)
k3 <- kmeans(students_features, centers=3, nstart=20)

> The `nstart` argument specifies a number of initial configurations and reports on the best one - an optimal number is usually somewhere between 20 and 50. (See more information in the Resources section - Professor Steorts, Duke University).

In [ ]:
# see components of k3
str(k3)

`kmeans` function returns the following components - among the most useful are:
- `cluster` - integers indicating the cluster assignment for each row
- `centers` - a matrix of cluster centers
- `totss` - the total sum of squares
- `withinss` - vector of within-cluster sum of squares, one component per cluster.
- `tot.withinss` - total within-cluster sum of squares, i.e. `sum(withinss)`
- `betweenss` - the between-cluster sum of squares, i.e. `totss-tot.withinss`
- `size` - the number of points in each cluster

As indicated above, the size of each cluster can be found by analyzing the `size` component.

In [ ]:
# see cluster size
k3$size

Most of the students are concentrated in clusters 1 and 3. In the perfect world, the students would be distributed more evenly across clusters, but oftentimes, it may make sense that they would not. Most importantly, the goal is for high intra-cluster similarity and low inter-cluster similarity.

To determine if `k=3` may accomplish the goal, basic descriptive statistics of the students within these clusters can be analyzed. To start the process, the clustering results can be added to the original data frame, `students` before finding the means of the various features within each cluster.

In [ ]:
 # add cluster number to the original dataframe and call frame_3
frame_3 <- data.frame(students_ml, k3$cluster)

# find within-cluster means
frame_3 %>% select(-drf_id) %>%
    group_by(k3.cluster) %>%
    summarize_all("mean")

Although cluster means should not be the only measure analyzed (e.g. standard deviation, median, etc.), the means can allow for a quick characterization of each of the clusters. How can these three clusters be described?

Cluster 2 and 3 are characterized by a higher number of semesters of funding in comparison with cluster 1. Cluster 2 is characterized by students who had a higher number of non-federal funding and cluster 3 is characterized by students who had, on the contrary, a higher number of federal funding. Cluster 1 is characterized by students who in general didn't have many semesters of any funding.

## Evaluate clusters

One simple way to evaluate resulting clusters is to compare the summary statistics between the key variables of interest.

The differences between the clusters can also be visualized in more detail by finding the mean and standard deviation of the variables. In this example, the variables of interest are primary source of support and time to degree completion from the SED data.

In [ ]:
# read into R
qry <- "
select drf_id, srceprim, ttddoc
from ds_nsf_ncses.dbo.nsf_sed
where phdfy = '2015'
"
sed <- dbGetQuery(con, qry)

In [ ]:
# add in more SED data to frame_3
frame_3 <- frame_3 %>% 
    inner_join(sed, by = c('drf_id'))

# head(frame_3)

In [ ]:
# Recode the primary source of support into the following categories, using case_when
frame_3 <- frame_3 %>%
            mutate(srceprim = case_when((srceprim == 'A' | srceprim == 'B') ~ 'Fellowship, scholarship or dissertation grant',
                                        srceprim == 'C' ~ 'Teaching assistantship',
                                      (srceprim == 'D' | srceprim == 'E' | srceprim == 'F' | srceprim == 'G') ~ 'Research assistantship or traineeship',
                                       (srceprim == 'H' | srceprim == 'I' | srceprim == 'J' | srceprim == 'K') ~ 'Own resources',
                                       (srceprim == 'L' | srceprim == 'M' | srceprim == 'N' | srceprim == '') ~ 'Other sources'))

The breakdown by primary source of support within each cluster can now be analyzed both numerically and visually.

> The proportions will be used since there are different counts within each of the clusters.

In [ ]:
# Proportions by sex within each cluster
cluster_counts_by_srceprim <- frame_3 %>%
    count(k3.cluster, srceprim) %>%
    group_by(k3.cluster) %>%
    mutate(prop = n/sum(n)) %>%
    select(-n) %>%
    ungroup()


cluster_counts_by_srceprim

In [ ]:
ggplot(cluster_counts_by_srceprim, aes(x = k3.cluster, y=prop, fill=srceprim)) +
    geom_bar(stat = 'identity', position = 'stack') +
    labs(
        x = "Cluster",
        y = "Proportion"
    )

The same process can be followed for the time to degree completion, with the addition of the standard deviation to account for the spread within each of the clusters for the numerical variable of interest (`ttddoc`).

In [ ]:
# average time to degree by cluster
time_to_degree <- frame_3 %>%
    group_by(k3.cluster) %>% 
    summarize(
        avg_time_to_degree = mean(ttddoc, na.rm = TRUE),
        sd = sd(ttddoc, na.rm = TRUE)
    )

time_to_degree

In [ ]:
ggplot(time_to_degree, aes(x=k3.cluster, y=avg_time_to_degree, fill=k3.cluster)) +
    geom_bar(stat="identity", position = position_dodge()) +   # plot bars for the mean values
    geom_errorbar(aes(ymax= avg_time_to_degree + sd, ymin = avg_time_to_degree-sd),            # add standard deviation bars
                  width=.2,
                  position = position_dodge(.9)) +
    labs(
        title = "No Significant Difference in TTD by Cluster",  # add title
        x = "Clusters",                                       
        y = "Mean"
    ) +                                              
    theme(text = element_text(size=16),                         # increase text font
          axis.text.x = element_text(size=18, face="bold"),     # increase text font on x-axis and make it bold
          legend.position = "none")                             # remove legend

It looks like federal vs non-federal funding doesn't matter for time to degree.

## Selecting *k*

Does it seem as though three clusters may be sufficient after quickly taking a look at the differentiations amongst these key variables? How can one be confident?

### Elbow method

The *Elbow method* can be used as one input in determining the optimal cluster number. Recall that *k*-means starts with k random cluster centers (centroids), assigns each data point to the closest centroid, and calculates the distances between each point and the centroid. Then, it moves the positions of the centroids and repeats the previous steps until there is convergence. In the *Elbow method*, the sum of squared errors (`SSE`) is calculated after the model converges for different values of *k*. All the resulting `SSE` values are plotted by increasing value of *k* in a line chart. The line chart should resemble an arm.

In [ ]:
set.seed(1)

# function to compute total within-cluster sum of square
wss <- function(k) {
    kmeans(students_features, k)$tot.withinss
}

# compute and plot wss for k =1 to k = 15
k.values <- 1:15

# extract wss values for each k
wss_values <- map_dbl(k.values, wss)

# plot the resulting SSE for each value of k
plot(k.values, wss_values, 
    type = "b", pch=19, frame=FALSE,
    xlab = "Number of clusters K", 
    ylab = "Total within-clusters sum of squares")

We can see that the SSE decreases as k increases. Here, the SSE decreases faster when k is small. As k increases, the reduction in SSE becomes smaller. Try to choose the number around the inflection point, where the change in SSE becomes negligible, indicating that there is little room to improve the model by increasing k (the bend in the elbow). On this graph, the elbow curve begins to flatten around k = 4.

Try running the model with 4 clusters.

In [ ]:
set.seed(1)
k4 <- kmeans(students_features, centers = 4)
k4$size

Save these results to a dataframe called `frame_4`, and check the characteristics of students in each cluster:

In [ ]:
frame_4 <- data.frame(students_ml, k4$cluster)  # add cluster number to the original dataframe

frame_4 %>% select(-drf_id) %>%
    group_by(k4.cluster) %>%
    summarize_all("mean")

Which clustering results - `frame_3` or `frame_4` - seem to be more useful for the data interpretation? Is there any additional information that should be considered?

In clustering, there is often no single right answer - everytime a different number of clusters is used, interesting patterns about the data can be exposed. However, it is crucial to think about and decide if whether the clusters represent true subgroups in the data. This could be a crucial input toward choosing the right number of clusters. (See more information on additional methods for selecting `k` in the Resources section - Professor Steorts, Duke University).

Experiment with different numbers of clusters in the Checkpoint below.

<font color=red><h3> Checkpoint: Run a K-Means clustering model </h3></font> 

1. Take a look again at the elbow curve, which number(s) of clusters look optimal?

2. Choose a different cluster number (other than 3 or 4). Use `kmeans()` to run a k-means clustering model with that number. Save the results and features in `frame_k`. 

3. Compare these results with the previous results. In which way the results are different?

4. Try comparing differences in clusters amongst other variables, beyond primary source of support and time to degree, to further justify if the selected k value may be optimal.

### Adding Categorical Variables

Clustering can also be performed using categorical variables. There are many ways to approach using a mix of categorical and numerical variables. These methods are all centered around the idea that there must be a distance measure that can incorporate both numerical and categorical variables. This can be done through either alternative distance metrics or through using categorical variables as numerical variables. 

In this example, we will go with the latter and the categorical variables will need to be converted to binary (0, 1) values and then scaled with the numerical variables. Note that this makes a pretty big assumption about how we should think about differences in categorical variables and differences in numerical variables. That is, we are assuming the maximum difference in any given numerical variable is the same as a difference on any individual indicator variable. Depending on the topic, this may or may not make sense. 

### Example: Modal funder

In [ ]:
# read into R
qry <- "
select sed.drf_id, team_size, any_non_federal, sem.modal_funder
from ds_nsf_ncses.dbo.nsf_sed sed
join tr_uncf_excelencia.dbo.sed_umetrics_xwalk xwalk
on sed.drf_id = xwalk.drf_id
join ds_iris_umetrics.dbo.semester sem
on xwalk.emp_number = sem.emp_number
where sed.phdfy = '2015'
"
students <- dbGetQuery(con, qry)

# see the dataframe
head(students)

Create the variables with the number of semesters of funding, federal and non-federal funding, like described at the beginning of the notebook:

In [ ]:
students <- students %>%
    group_by(drf_id) %>%
    mutate(sem_funding = n()) 

In [ ]:
# If team_size > 0, then flag as federal (add 1), if team_size is missing, then add 1 in the "any_non_federal" variable
students <- students %>%
    mutate(federal = ifelse(team_size > 0, 1, 0),
           non_federal = ifelse(is.na(team_size) | any_non_federal == 1, 1, 0))

In [ ]:
students <- students %>%
            group_by(drf_id) %>%                                 # for each unique drf_id
            mutate(sem_federal = sum(federal, na.rm = TRUE),     # sum values in the federal flag
                   sem_non_federal = sum(non_federal)) %>%       # sum values in the any_non_federal flag
            select(-c(team_size, any_non_federal, federal, non_federal)) %>%  # remove these columns
            slice(1) %>% ungroup()                                         # deduplicate rows, choose just 1 row per unique drf_id

Segment the data frame by 5 major agencies (NIH, NSF, DOE, DOD, USDA), otherwise overwriting `modal_funder` as `"OTHER"`.

In [ ]:
students <- students %>%
    mutate(modal_funder = ifelse(modal_funder %in% c("NIH", "NSF", "DOE", "DOD", "USDA"), modal_funder, "OTHER"))

In [ ]:
students <- students %>%
            replace(is.na(.), 0)

In [ ]:
modal_funder_df <- students %>%
                    select(c(drf_id, modal_funder))

In [ ]:
# Use dcast function to get create 0 and 1 values for the primary source of support variable
modal_funder_df <- dcast(data = modal_funder_df, drf_id ~ modal_funder, length)

merged <- inner_join(students, modal_funder_df, by=c('drf_id'))

In [ ]:
head(merged)

In [ ]:
merged <- merged %>%
            select(-c(drf_id, modal_funder))

Show the head of the table with modal funder coded as 0 and 1:

In [ ]:
head(merged)

In [ ]:
# Scale the features
merged_scale <- scale(merged)

# View first rows after scaling
head(merged_scale)

In [ ]:
# Remove all missing data points before running clustering
# na.omit will remove any rows with any NA values
merged_scale <- na.omit(merged_scale)

In [ ]:
merged_scale <- as.data.frame(merged_scale)

In [ ]:
set.seed(1)

# function to compute total within-cluster sum of square
wss <- function(k) {
    kmeans(merged_scale, k)$tot.withinss
}

# compute and plot wss for k =1 to k = 15
k.values <- 1:15

# extract wss values for each k
wss_values <- map_dbl(k.values, wss)

# plot the resulting SSE for each value of k
plot(k.values, wss_values, 
    type = "b", pch=19, frame=FALSE,
    xlab = "Number of clusters K", 
    ylab = "Total within-clusters sum of squares")

Based on the Elbow method, it looks like 7 clusters are optimal.

In [ ]:
# Initialize the model 
set.seed(1)
k7 <- kmeans(merged_scale, centers=7, nstart=20)

In [ ]:
merged <- na.omit(merged)                     # remove missing values
frame_7 <- data.frame(merged, k7$cluster)  # add cluster number to the original dataframe 

frame_7 %>% 
    group_by(k7.cluster) %>%
    summarize_all("mean")

How might you interpret this? Can you identify clusters of interest? 

<font color=red><h3> Checkpoint: Adding more variables </h3></font> 

Try adding more variables. For example, you can try including all of the variables used above to generate clusters. If you added features based on topics in the text analysis in the previous checkpoint, you can include those here as well. What do each of the clusters represent? Note that the interpretation will be harder with more features.

Then, look at the distributions of other variables, such as primary source of support and race within each of the clusters. What would be your conclusions based on these distributions and your interpretation of the clusters?

In [ ]:
# Close the database connection
dbDisconnect(con)

### Resources:
- UC Business Analytics R Programming Guide: https://uc-r.github.io/kmeans_clustering
- Rebecca Steorts, Assistant Professor, Duke University, Department of Statistical Science, Data Mining and Machine Learning course: https://github.com/resteorts/data-mine/tree/master/lectures_2018/10-unsupervise/10-kmeans.pdf